# OpenScope Simple Demo (Refactored)

This notebook demonstrates the simplified OpenScope RL environment using the refactored codebase.

## What This Demo Does:
1. Uses the refactored `PlaywrightEnv` environment
2. Retrieves game state using the modular `OpenScopeInterface`
3. Issues random commands using the environment's action space
4. Shows how the refactored code simplifies interaction

## Prerequisites:
- OpenScope server running on http://localhost:3003
- Refactored environment modules installed
- Playwright installed (`pip install playwright`)
- Browsers installed (`playwright install chromium`)
- nest_asyncio installed (`pip install nest-asyncio`) for Jupyter compatibility


## Setup and Imports


In [2]:
# Apply nest_asyncio FIRST, before ANY other imports
import nest_asyncio
nest_asyncio.apply()
print("✅ nest_asyncio applied for Jupyter compatibility")

import random
import asyncio
from environment import PlaywrightEnv, create_default_config
from environment.utils import BrowserManager
from environment.constants import DEFAULT_GAME_URL, DEFAULT_AIRPORT, DEFAULT_TIMEWARP

print("✅ Imports complete")
print(f"   Game URL: {DEFAULT_GAME_URL}")
print(f"   Airport: {DEFAULT_AIRPORT}")
print(f"   Timewarp: {DEFAULT_TIMEWARP}x")
print("   Using refactored environment modules")


✅ nest_asyncio applied for Jupyter compatibility
✅ Imports complete
   Game URL: http://localhost:3003
   Airport: KLAS
   Timewarp: 5x
   Using refactored environment modules


## Initialize Environment


In [3]:
# Create environment configuration
config = create_default_config(
    max_aircraft=10,
    episode_length=300,
    headless=False  # Set to True for headless mode
)

print("🔧 Environment Configuration:")
print(f"   Max aircraft: {config.max_aircraft}")
print(f"   Episode length: {config.episode_length}")
print(f"   Headless: {config.browser_config.headless}")
print(f"   Airport: {config.airport}")
print(f"   Timewarp: {config.timewarp}x")


🔧 Environment Configuration:
   Max aircraft: 10
   Episode length: 300
   Headless: False
   Airport: KLAS
   Timewarp: 5x


## Create and Initialize Environment


In [4]:
# Create the environment
env = PlaywrightEnv(
    max_aircraft=config.max_aircraft,
    episode_length=config.episode_length,
    headless=config.browser_config.headless,
    airport=config.airport,
    timewarp=config.timewarp
)

print("🌐 Environment created successfully")
print(f"   Observation space: {env.observation_space}")
print(f"   Action space: {env.action_space}")


🌐 Environment created successfully
   Observation space: Dict('aircraft': Box(-inf, inf, (10, 14), float32), 'aircraft_mask': Box(False, True, (10,), bool), 'conflict_matrix': Box(0.0, 1.0, (10, 10), float32), 'global_state': Box(-inf, inf, (4,), float32))
   Action space: Dict('aircraft_id': Discrete(11), 'altitude': Discrete(18), 'command_type': Discrete(5), 'heading': Discrete(13), 'speed': Discrete(8))


## Reset Environment and Get Initial State


In [5]:
# Reset environment to start a new episode
obs, info = env.reset()

print("🎮 Environment reset successfully")
print(f"   Observation keys: {list(obs.keys())}")
print(f"   Aircraft data shape: {obs['aircraft'].shape}")
print(f"   Aircraft mask shape: {obs['aircraft_mask'].shape}")
print(f"   Conflict matrix shape: {obs['conflict_matrix'].shape}")
print(f"   Global state shape: {obs['global_state'].shape}")
print(f"   Info keys: {list(info.keys())}")


🎮 Environment reset successfully
   Observation keys: ['aircraft', 'aircraft_mask', 'global_state', 'conflict_matrix']
   Aircraft data shape: (10, 14)
   Aircraft mask shape: (10,)
   Conflict matrix shape: (10, 10)
   Global state shape: (4,)
   Info keys: ['raw_state', 'episode_metrics', 'config']


## Take Random Actions


In [6]:
# Take a few random actions to demonstrate the environment
print("🎯 Taking random actions...")

for step in range(5):
    # Sample a random action
    action = env.action_space.sample()
    
    print(f"\nStep {step + 1}:")
    print(f"   Action: {action}")
    
    # Execute the action
    obs, reward, terminated, truncated, info = env.step(action)
    
    print(f"   Reward: {reward:.3f}")
    print(f"   Terminated: {terminated}")
    print(f"   Truncated: {truncated}")
    print(f"   Score: {info.get('score', 'N/A')}")
    print(f"   Aircraft count: {info.get('aircraft_count', 'N/A')}")
    
    if terminated or truncated:
        print("   Episode ended!")
        break

print("\n✅ Random actions completed")


🎯 Taking random actions...

Step 1:
   Action: {'aircraft_id': np.int64(5), 'altitude': np.int64(6), 'command_type': np.int64(1), 'heading': np.int64(11), 'speed': np.int64(6)}
   Reward: 0.090
   Terminated: False
   Truncated: False
   Score: 0
   Aircraft count: 14

Step 2:
   Action: {'aircraft_id': np.int64(7), 'altitude': np.int64(0), 'command_type': np.int64(0), 'heading': np.int64(2), 'speed': np.int64(4)}
   Reward: 0.090
   Terminated: False
   Truncated: False
   Score: 0
   Aircraft count: 14

Step 3:
   Action: {'aircraft_id': np.int64(2), 'altitude': np.int64(11), 'command_type': np.int64(2), 'heading': np.int64(3), 'speed': np.int64(6)}
   Reward: 0.090
   Terminated: False
   Truncated: False
   Score: 0
   Aircraft count: 14

Step 4:
   Action: {'aircraft_id': np.int64(10), 'altitude': np.int64(3), 'command_type': np.int64(0), 'heading': np.int64(2), 'speed': np.int64(4)}
   Reward: 0.090
   Terminated: False
   Truncated: False
   Score: 0
   Aircraft count: 14

Step 

## Manual Command Execution (Optional)


In [7]:
# You can also execute manual commands using the environment's interface
print("🎮 Manual command execution example:")

# Get current game state
game_state = env.game_interface.get_game_state()

if game_state and game_state.get('aircraft'):
    print(f"\n📋 Current Aircraft ({len(game_state['aircraft'])} total):")
    for i, ac in enumerate(game_state['aircraft'][:5]):  # Show first 5
        print(f"   {i+1}. {ac['callsign']}:")
        print(f"      Altitude: {ac['altitude']:.0f} ft")
        print(f"      Heading: {ac['heading']:.0f}°")
        print(f"      Speed: {ac['speed']:.0f} kts")
        print(f"      Category: {ac['category']}")
    
    if len(game_state['aircraft']) > 5:
        print(f"   ... and {len(game_state['aircraft']) - 5} more aircraft")
    
    print(f"\n📊 Game State:")
    print(f"   Score: {game_state.get('score', 'N/A')}")
    print(f"   Time: {game_state.get('time', 'N/A')}")
    print(f"   Conflicts: {len(game_state.get('conflicts', []))}")
else:
    print("❌ No aircraft in the game")


🎮 Manual command execution example:

📋 Current Aircraft (14 total):
   1. DAL73:
      Altitude: 13000 ft
      Heading: 1°
      Speed: 280 kts
      Category: arrival
   2. NKS3194:
      Altitude: 24000 ft
      Heading: 1°
      Speed: 280 kts
      Category: arrival
   3. NKS2625:
      Altitude: 11000 ft
      Heading: 4°
      Speed: 280 kts
      Category: arrival
   4. NKS7852:
      Altitude: 17000 ft
      Heading: 4°
      Speed: 280 kts
      Category: arrival
   5. N227BT:
      Altitude: 19000 ft
      Heading: 4°
      Speed: 280 kts
      Category: arrival
   ... and 9 more aircraft

📊 Game State:
   Score: 0
   Time: 3.0175
   Conflicts: 0


## Enhanced State Extraction (Optional)


In [8]:
# You can also get enhanced state with additional properties
print("🔍 Enhanced state extraction:")

try:
    from environment.utils import extract_enhanced_game_state
    
    enhanced_state = extract_enhanced_game_state(env.browser_manager.page)
    
    if enhanced_state and enhanced_state.get('aircraft'):
        print(f"\n✅ Enhanced State Retrieved:")
        print(f"   Total aircraft: {enhanced_state['numAircraft']}")
        print(f"   Active conflicts: {len(enhanced_state['conflicts'])}")
        print(f"   Current score: {enhanced_state['score']}")
        print(f"   Game time: {enhanced_state['time']}")
        
        if enhanced_state.get('weather'):
            print(f"   Weather data: {list(enhanced_state['weather'].keys())}")
        
        # Show detailed info for first aircraft
        if enhanced_state['aircraft']:
            ac = enhanced_state['aircraft'][0]
            print(f"\n📋 Detailed Aircraft Info ({ac['callsign']}):")
            
            # Show some additional properties if available
            additional_props = [
                ('verticalSpeed', 'Vertical Speed'),
                ('flightPhase', 'Flight Phase'),
                ('squawk', 'Squawk'),
                ('targetRunway', 'Target Runway'),
                ('windSpeed', 'Wind Speed'),
                ('mach', 'Mach Number')
            ]
            
            for prop, label in additional_props:
                if ac.get(prop) is not None:
                    print(f"   {label}: {ac[prop]}")
    else:
        print("❌ Could not retrieve enhanced game state")
        
except Exception as e:
    print(f"❌ Enhanced state extraction failed: {e}")


🔍 Enhanced state extraction:
❌ Enhanced state extraction failed: cannot import name 'extract_enhanced_game_state' from 'environment.utils' (/Users/jmzlx/Projects/atc/openscope-rl/environment/utils.py)


## Cleanup


In [9]:
print("🧹 Cleaning up...")

# Close the environment
env.close()

print("✅ Environment closed successfully")
print("\n🎉 Demo completed!")
print("\nThe refactored environment provides:")
print("   • Clean separation of concerns")
print("   • Modular JavaScript functions")
print("   • Type-safe configuration")
print("   • Comprehensive error handling")
print("   • Easy-to-use interface")


🧹 Cleaning up...
✅ Environment closed successfully

🎉 Demo completed!

The refactored environment provides:
   • Clean separation of concerns
   • Modular JavaScript functions
   • Type-safe configuration
   • Comprehensive error handling
   • Easy-to-use interface
